# 연습문제

2. 청와대 국민 청원 게시판에 있는 다양한 청원들의 “전체 목록” 에서 각 청원에 대한 상세 내역을 수집하는 웹 크롤러를 만드세요. 수집된 데이터 형식은 다음 페이지에 있는 각 형식별 수집 예시를 참고하세요

In [3]:
from selenium import webdriver
import time 
import os
import time
import pandas as pd

print("=" *100)
print("청와대 국민 청원 게시판에 있는 다양한 청원들의 “전체 목록” 에서 각 청원에 대한 상세 내역을 수집")
print("=" *100)

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"
# 저장될 파일위치와 이름을 지정합니다

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

query_txt = '청와대국민청원'
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'https://www1.president.go.kr/petitions'
driver.get(url)
driver.maximize_window()
time.sleep(2)

from bs4 import BeautifulSoup
import math

cnt = int(input('크롤링할 건수 입력:'))
page_cnt = math.ceil(cnt/7)
print('크롤링 할 총 페이지 번호: ', page_cnt)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(cnt, page_cnt))
print("\n")

# url을 따와야 함
url_list = []
count = 0

for i in range(1, page_cnt+1):
    html_url = driver.page_source
    soup_url = BeautifulSoup(html_url, 'html.parser')
    
    url1 = soup_url.find_all('div', 'board text')
    url2 = url1[2].find('div', 'b_list category b_list2')
    url3 = url2.find('div', 'bl_body').find('ul', 'petition_list').find_all('li')
    
    for j in url3:
        url_ = j.find('a')['href']
        full_url = 'https://www1.president.go.kr'+url_
        url_list.append(full_url)
        
        count += 1

        if count == cnt :
            break
            
    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(i,count))
    print("\n")
    
    i += 1
    try:
        driver.find_element_by_link_text('%s' %i).click()
    except:
        driver.find_element_by_link_text('Next').click()
    time.sleep(2)

no = 1

cno2=[]
title2=[]
people2=[]
category2=[]
s_date2=[]
e_date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no)
    no += 1
    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 저장한 url 원소의 html 파싱 정보를 가져옴

    content_all = soup.find('div','petitionsView_left_pg')
    
    f = open(ff_name, 'a',encoding='UTF-8')

    # 청원 등록 번호
    c_no = url_list[i].split('/')
    #cno_1 = c_no[4].split('?')
    cno_1 = c_no[4]        
    print('1.청원글번호: ',cno_1 ,'\n')
    f.write('\n')
    f.write("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no + '\n')
    f.write('1.청원번호:' + cno_1 + '\n')
    cno2.append(cno_1)

    # 청원제목 추출
    title = content_all.find('h3','petitionsView_title').get_text()
    print('2.청원제목: ' , title , '\n')
    f.write('2.청원제목:' + title + '\n')
    title2.append(title)

    # 참여인원 추출
    people = content_all.find('h2','petitionsView_count').find('span','counter').get_text()
    print('3.참여인원: ' , people ,'명' ,'\n')
    f.write('3.참여인원:' + people + '\n')
    people2.append(people)

    # 카테고리
    cat_all = content_all.find('div','petitionsView_info').find('ul','petitionsView_info_list').find_all('li')

    category = cat_all[0].get_text().replace('카테고리','')
    print('4.카테고리: ',category ,'\n')
    f.write('4.카테고리:' + category + '\n')
    category2.append(category)

    # 청원시작일
    s_date = cat_all[1].get_text().replace('청원시작','')
    print('5.청원시작일: ',s_date ,'\n')
    f.write('5.청원시작일:' + s_date + '\n')
    s_date2.append(s_date)

    # 청원마감일
    e_date = cat_all[2].get_text().replace('청원마감','')
    print('6.청원마감일: ',e_date ,'\n')
    f.write('6.청원종료일:' + e_date + '\n')
    e_date2.append(e_date)

    # 청원내용
    content = content_all.find('div','View_write').get_text().replace('\n','')
    print('7.청원내용: ', content.strip() , '\n')
    f.write('7.청원내용:' + content.strip() + '\n')
    content2.append(content)

    f.close( )
    
    # Step 6. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

blue_house = pd.DataFrame()

blue_house['청원번호'] = cno2
blue_house['청원제목'] = pd.Series(title2) 
blue_house['참여자'] = pd.Series(people2)
blue_house['카테고리'] = pd.Series(category2)
blue_house['청원시작일'] = pd.Series(s_date2)
blue_house['청원종료일'] = pd.Series(e_date2)
blue_house['청원내용'] = pd.Series(content2)
       
# csv 형태로 저장하기
blue_house.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
blue_house.to_excel(fx_name , index=False)

driver.close( )

청와대 국민 청원 게시판에 있는 다양한 청원들의 “전체 목록” 에서 각 청원에 대한 상세 내역을 수집
2.파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):
크롤링할 건수 입력:10
크롤링 할 총 페이지 번호:  2
10 건의 데이터를 수집하기 위해 2 페이지의 게시물을 조회합니다.


1 번째 페이지까지 7건 정보 수집 완료 ====================


2 번째 페이지까지 10건 정보 수집 완료 ====================




1 번째 국민청원 게시글 상세 정보입니다==========================
['https:', '', 'www1.president.go.kr', 'petitions', '599809']
1.청원글번호:  599809 

2.청원제목:  (얀센) 형부_백신접종 후 사지마비, 결국 중환자실 

3.참여인원:  3,061 명 

4.카테고리:  보건복지 

5.청원시작일:  2021-07-13 

6.청원마감일:  2021-08-12 

7.청원내용:  (얀센) 백신접종 후 사지마비, 결국 중환자실 현재 이 상황에 처한 언니는 이루 말할 수 없는 충격에 이 글을 작성할 수 없기에 동생인 제가 대신 적어보려 합니다. 저희 형부가 얀센백신 접종(2021년 06월 18일)후 약 10일후부터 이상반응이 오기 시작했습니다. 그 전까지는 약간의 피곤함 무기력함 등 거의 무증상으로 일상생활이 가능했고 늘상 해오던 본인의 업도 수행 가능할 만큼 건강 했습니다. 특이 기저질환 및 병력도 없었을 뿐더러, 키는 무려 187㎝에 몸무게 90kg의 건강한 체구로, 덩치 큰 두 아들(5살, 8살)을 두 팔로 동시에 번쩍번쩍 들 만큼 건강하고 건장 했던 40대 초반의 남자였습니다. 문제는 접종 후 10일 이후(21년 06월 28일)부터 였다고 합니다. 오전부터 손발 저리는 증상 발생과 함께 깨질 듯 한 두통이 시작되었지만 그래도 이때는 그나마 완전한 보행이 가능했습니다. 이틀 후(21년 6월 30일부터) 상태는 더욱 악화 되

['https:', '', 'www1.president.go.kr', 'petitions', '599806']
1.청원글번호:  599806 

2.청원제목:  비민주적으로 혁신학교를 강행하려는 **시 **중학교 교장을 규탄합니다. 

3.참여인원:  706 명 

4.카테고리:  육아/교육 

5.청원시작일:  2021-07-13 

6.청원마감일:  2021-08-12 

7.청원내용:  안녕하세요, 문재인 대통령님.나라와 국민을 위한 자주적이고 민주적인 대한민국 건설에 하루 하루 고생이 많으십니다.저는 여지껏 청와대 게시판 직접 들어와서 글 한 번 써본적 없는 학부모입니다.제가 알고있는 연락처로 연락은 했으나 여전히 명확하지 않고 공개되지 않은 정보와 절차로 민주적이라고 하면서 **시 **중학교를 혁신학교로 전환하려고 하는 **중학교 교장/교감선생님의 비민주적인 행동에 대해 누구에게 하소연을 할 수 없어 이 곳에 글을 적습니다.다음은 제가 **중학교 교감선생님과 통화한 내용 요약입니다.1. **중학교측은 혁신학교 관련 학무모님들을 참여하실 수 있게 Zoom으로 여러번 연수를 하였다고 하나, 실제로 해당 설명회 또는 연수회의 참석율이나 참석인원에 대해서는 밝히지 않고 있습니다. 2. 대다수의 관련 학부모들은 이에 대해 반대를 하고 있는데, 투표율 또한 공개하지 않으며 총인원 몇 명 중 몇 명이 찬성/반대하였는지도 공개하지 않고 있습니다. 저조한 투표율이었는지 동의율만 절반정도라고 답변합니다. 3. 아이들은 위한 교육에 더 많은 예산을 받을 수 있어 좋다고 하셨는데 실제로 혁신학교 전환이후 학업성취도는 저하되었고 이에 대해 객관적인 데이터나 논리로 반박하지는 못하십니다.4. 학운위의 대표성에 대해서도 반문하였으나, 해당건은 본인이 담당이 아니라 행정실에 문의하시고 학운위 일정은 모르겠다라고 하십니다. 학운위는 실제로 투표가 아닌 일방 통보로 구성되었습니다. 중등부 담당 장학사는 현재 코로나대응팀에 있어 답변도 못받았습니다.저는 이러한 민원처리에 대해 분노합니

['https:', '', 'www1.president.go.kr', 'petitions', '599801']
1.청원글번호:  599801 

2.청원제목:  아빠는 코로나19 양성이지만 지금도 아들과 집에있습니다. 

3.참여인원:  242 명 

4.카테고리:  보건복지 

5.청원시작일:  2021-07-13 

6.청원마감일:  2021-08-12 

7.청원내용:  저는 부득이한 사정으로 남편과 아이와 떨어져지내고 있습니다.7.10 토요일 남편의 코로나19 양성소식에 하늘이 무너졌습니다.아이는 검사결과 음성으로 확인되었지만 자가격리중이고또 검사를 해야하는 상황입니다.토요일 일요일 오늘 현재까지 **보건소는 병실이 없다는이유로 아이와 아빠를 한집에 방치하고 있습니다.어제 뉴스기사에서는 병상대기하는 환자가 없다고 보았는데기사와 현실은 정말 다른상황이네요병실이 부족한 현실이라면 모든 국민에게 빨리 정확하게 알리시고 빠른 방법을 강구해주시기를 부탁드립니다. [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다] 



10 번째 국민청원 게시글 상세 정보입니다==========================
['https:', '', 'www1.president.go.kr', 'petitions', '599800']
1.청원글번호:  599800 

2.청원제목:  어린이집 긴급보육 개선 촉구 

3.참여인원:  506 명 

4.카테고리:  보건복지 

5.청원시작일:  2021-07-13 

6.청원마감일:  2021-08-12 

7.청원내용:  유, 초, 중, 고를 담당하는 교육부는 정부의 방역지침에 따라 신속히 단계별 방역대응을 실시하고 있으나, 보건복지부 산하 어린이집은 명확한 단계별 대응 없이 영유아들의 건강을 오롯이 어린이집에 그 책임을 떠맡기고 있습니다.재택근무와 원격수업의 의미는 최대한 타인과의 접촉을 줄이고 소그룹 버블 방역을 하기 위함인데, 어린이집은 이러한 기본적인 방역의 환경에서 철저히 소외당하고 있습니다

<ipython-input-3-e7bd01422deb>:169: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  blue_house.to_excel(fx_name , index=False)


In [23]:
driver.get(url_list[0])
html_url = driver.page_source
soup_url = BeautifulSoup(html_url, 'html.parser')

print(url_list[0])

https://www1.president.go.kr/petitions/599809
